In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/Users/arnaud/Desktop/HEC/taxi-time/Data/Airport_Data.csv")

In [24]:
df.columns.shape

(43,)

In [100]:
columns_df = pd.read_excel("/Users/arnaud/Desktop/HEC/taxi-time/Data/Glossary.xlsx", sheet_name = 1).iloc[:,[0,2]]

In [101]:
columns_df.iloc[3, 1]

'Aircraft Type (with another regulation -not to be used for the case)'

In [102]:
#columns_df[columns_df.iloc[:,1] not in  ['To ignore', 'to ignore']]

In [103]:
#columns_df.iloc[:,0] not in  ['To ignore', 'to ignore']

In [105]:
columns_df

,English,Meaning
0,Carrier,Airline company
1,Flight,Flight number (e.g. AF1196)
2,acType,Aircraft type
3,acReg,Aircraft Type (with another regulation -not to...
4,Ship,Shipment of the plane
5,mode-s,Technical characteristic to ignore
6,Status,Technical characteristic to ignore
7,sto,Scheduled time-off
8,sqt,Technical characteristic to ignore
9,runway,"Long ""road"" on which the airplane lands or tak..."


In [106]:
[columns_df.iloc[:,0].tolist()[i] for i in range((len(columns_df.iloc[:,1].tolist()))) if columns_df.iloc[:,1].tolist()[i] not in ['To ignore', 'to ignore',  'Technical characteristic to ignore', 'Aircraft Type (with another regulation -not to be used for the case)'] ]



['Carrier',
 'Flight',
 'acType',
 'Ship',
 'sto',
 'runway',
 'Stand',
 'ALDT',
 'CIBT',
 'Chock_on',
 'EIBT',
 'CIBT',
 'AIBT']

In [75]:
columns_df.iloc[:,1].tolist()[2]

'Aircraft'

In [107]:
def glossary_feature_selection(path):
    ignore = ['To ignore', 'to ignore',  'Technical characteristic to ignore', 'Aircraft Type (with another regulation -not to be used for the case)']
    columns_df = pd.read_excel(path, sheet_name = 1).iloc[:,[0,2]]
    feature_list = [columns_df.iloc[:,0].tolist()[i] for i in range((len(columns_df.iloc[:,1].tolist()))) if columns_df.iloc[:,1].tolist()[i] not in ignore ]
    return feature_list

    
    

In [108]:
glossary_feature_selection("/Users/arnaud/Desktop/HEC/taxi-time/Data/Glossary.xlsx")

['Carrier',
 'Flight',
 'acType',
 'Ship',
 'sto',
 'runway',
 'Stand',
 'ALDT',
 'CIBT',
 'Chock_on',
 'EIBT',
 'CIBT',
 'AIBT']